In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model-till-32000/config.json
/kaggle/input/model-till-32000/merges.txt
/kaggle/input/model-till-32000/vocab.json
/kaggle/input/model-till-32000/tokenizer_config.json
/kaggle/input/model-till-32000/model.safetensors
/kaggle/input/model-till-32000/special_tokens_map.json
/kaggle/input/model-till-32000/added_tokens.json
/kaggle/input/model-till-32000/generation_config.json
/kaggle/input/news-2-0/news_2.0.csv
/kaggle/input/model-till-25000/config.json
/kaggle/input/model-till-25000/merges.txt
/kaggle/input/model-till-25000/vocab.json
/kaggle/input/model-till-25000/tokenizer_config.json
/kaggle/input/model-till-25000/model.safetensors
/kaggle/input/model-till-25000/special_tokens_map.json
/kaggle/input/model-till-25000/added_tokens.json
/kaggle/input/model-till-25000/generation_config.json


In [15]:
from transformers import pipeline

import matplotlib.pyplot as plt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [16]:
# df = pd.read_csv("/kaggle/input/news-2-0/news_2.0.csv")

In [17]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import pipeline

model_ckpt = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipeline('summarization', model=model_ckpt, device = device)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [18]:
# Example function to generate segments of text within the maximum sequence length
def generate_segments(input_text):
    words = input_text.split()
    for i in range(0, len(words), 300):  # Adjust the segment length here
        yield ' '.join(words[i:i+300])   # Adjust the segment length here

def summarization(input_text):
    global count
    if(len(tokenizer.tokenize(input_text))<=1020):
        count += 1
        print(count, end=" ")
        return input_text

    output_summary = ""
    for segment in generate_segments(input_text):
        try:
            max_length = len(segment.split()) # Adjust max_length dynamically
            summarized_segment = pipe(segment, max_length=max_length, min_length=1, do_sample=False)[0]['summary_text']
            output_summary += summarized_segment
            
        except IndexError:
            pass
        except Exception as e:
            print(f"Error occurred: {e}")
    if len(output_summary.split()) >= 1020:
            output_summary = summarization(output_summary)
    count += 1
    print(count, end=" ")
    return output_summary
count = 0

In [43]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "/kaggle/input/model-till-12000"

# Load the model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.add_special_tokens({"pad_token":"<pad>"})
model = GPT2LMHeadModel.from_pretrained(model_path)
model.resize_token_embeddings(len(tokenizer))

# If GPU is available, move the model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50258, bias=False)
)

In [20]:
!pip install flask-ngrok >> /dev/null
!pip install flask_cors
!pip install pyngrok
from flask import Flask,request,jsonify
from pyngrok import ngrok
from flask_cors import CORS
from flask import Response
!ngrok config add-authtoken 2cDjv295789xMxfOXzNy20lFY2C_2zrPAEF32U1teXtTXx2rr
ngrok.set_auth_token("2cDjv295789xMxfOXzNy20lFY2C_2zrPAEF32U1teXtTXx2rr")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [45]:
def generation(query):
    headline = []
    input_ids = tokenizer(query, return_tensors='pt')
    # Move the input to GPU if available
    input_ids = input_ids.to("cuda")

    # Generate 5 different outputs
    num_outputs = 7
    generated_outputs = []
    max_new_tokens = 24
    for _ in range(num_outputs):
        # Generate output using the model
        output = model.generate(
            input_ids=input_ids["input_ids"],
            attention_mask=input_ids['attention_mask'],
            length_penalty=0.8,
            min_new_tokens=5,
            max_new_tokens=max_new_tokens+_,
            num_beams=8,
            no_repeat_ngram_size=2,
            early_stopping=True,
            do_sample=True
        )
        generated_outputs.append(tokenizer.decode(output[0]))
    for i, output_text in enumerate(generated_outputs):
        # Remove <pad> tokens
        output_text = output_text.replace("<pad>", "")
        output_text = output_text.replace("<|endoftext|>", "")

        # Find the index of "TL;DR:"
        tldr_index = output_text.find("TL;DR:")
        if tldr_index != -1:
            # Extract the text after "TL;DR:"
            output_text = output_text[tldr_index + len("TL;DR:"):]

        # Remove commas from the output text
        output_text = output_text.replace(",", "")
        headline.append(output_text.strip())
    return headline
    

In [ ]:
app = Flask(__name__)
CORS(app)

@app.before_request
def basic_authentication():
    if request.method.lower() == 'options':
        return Response()


@app.route("/")
def home():
    return jsonify({'result': "result"})

@app.route('/api/data/',methods=['POST'])
def get_data():
    article = request.get_json()
    sentence = summarization(article)
    query = sentence + '\nTL;DR:'
    result = generation(query)
    return result

ngrok_tunnel = ngrok.connect(5000)
print(' * Tunnel URL:', ngrok_tunnel.public_url)

if __name__ == "__main__":
    app.run()

 * Tunnel URL: https://c2ba-34-73-219-99.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


26 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


27 

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


34 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


35 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
